# Binary Classification Model for Disease Prediction


# Part 1: Installation of necessary libraries

In [57]:
import numpy as np
import pandas as pd
import os 
import random
import tensorflow
from scipy.spatial import distance
import time


# Part 2: Download of patient data with corresponding disease status

In [105]:
pd.set_option("display.max_columns", None) # This ensures the full dataframe is visulised and not truncated
disease_data = pd.read_csv('Training.csv')

# Part 3 : Exploratory Data Analysis


In [106]:
disease_data['EJ'] = disease_data['EJ'].map({'A': 0, 'B': 1})
print(disease_data)

               Id        AB          AF          AH          AM         AR   
0    000ff2bfdfe9  0.209377  3109.03329   85.200147   22.394407   8.138688  \
1    007255e47698  0.145282   978.76416   85.200147   36.968889   8.138688   
2    013f2bd269f5  0.470030  2635.10654   85.200147   32.360553   8.138688   
3    043ac50845d5  0.252107  3819.65177  120.201618   77.112203   8.138688   
4    044fb8a146ec  0.380297  3733.04844   85.200147   14.103738   8.138688   
..            ...       ...         ...         ...         ...        ...   
612  fd3dafe738fd  0.149555  3130.05946  123.763599    9.513984  13.020852   
613  fd895603f071  0.435846  5462.03438   85.200147   46.551007  15.973224   
614  fd8ef6377f76  0.427300  2459.10720  130.138587   55.355778  10.005552   
615  fe1942975e40  0.363205  1263.53524   85.200147   23.685856   8.138688   
616  ffcca4ded3bb  0.482849  2672.53426  546.663930  112.006102   8.138688   

           AX        AY         AZ          BC         BD      

In [107]:

disease_data=disease_data.dropna()
print(disease_data)

               Id        AB          AF          AH          AM         AR   
0    000ff2bfdfe9  0.209377  3109.03329   85.200147   22.394407   8.138688  \
1    007255e47698  0.145282   978.76416   85.200147   36.968889   8.138688   
2    013f2bd269f5  0.470030  2635.10654   85.200147   32.360553   8.138688   
3    043ac50845d5  0.252107  3819.65177  120.201618   77.112203   8.138688   
4    044fb8a146ec  0.380297  3733.04844   85.200147   14.103738   8.138688   
..            ...       ...         ...         ...         ...        ...   
611  fd1dd68d51b4  0.175193  2607.26686   85.200147    7.067354   8.138688   
612  fd3dafe738fd  0.149555  3130.05946  123.763599    9.513984  13.020852   
613  fd895603f071  0.435846  5462.03438   85.200147   46.551007  15.973224   
614  fd8ef6377f76  0.427300  2459.10720  130.138587   55.355778  10.005552   
616  ffcca4ded3bb  0.482849  2672.53426  546.663930  112.006102   8.138688   

           AX        AY         AZ          BC         BD      

In [108]:
print(disease_data.iloc[:,-1])
print(disease_data.iloc[0,1:-1])
print(disease_data.iloc[1,0:])

0      1
1      0
2      0
3      0
4      1
      ..
611    0
612    0
613    0
614    0
616    0
Name: Class, Length: 548, dtype: int64
AB        0.209377
AF      3109.03329
AH       85.200147
AM       22.394407
AR        8.138688
AX        0.699861
AY        0.025578
AZ        9.812214
BC        5.555634
BD      4126.58731
BN         22.5984
BP      175.638726
BQ      152.707705
BR      823.928241
BZ      257.432377
CB       47.223358
CC        0.563481
CD         23.3876
CF        4.851915
CH        0.023482
CL        1.050225
CR        0.069225
CS       13.784111
CU        1.302012
CW       36.205956
DA         69.0834
DE      295.570575
DF         0.23868
DH        0.284232
DI        89.24556
DL        84.31664
DN       29.657104
DU         5.31069
DV         1.74307
DY       23.187704
EB        7.294176
EE        1.987283
EG      1433.16675
EH        0.949104
EJ               1
EL        30.87942
EP       78.526968
EU        3.828384
FC        13.39464
FD       10.265073
FE     

In [109]:
def euclidean_distance(sample1, sample2):
    """Calculate the Euclidean distance between two samples."""
    return distance.euclidean(sample1, sample2)

# Part 4 : Generating New Data Features

In [121]:


def euclidean_feature_generator(Sample_List):
    euclidean_feature_dataframe=pd.DataFrame()
    
    for Sample in range(0,len(Sample_List)):#0:640
        SamplePairsData=[]

        for Alternate_Sample in range(0,len(Sample_List)):
            SamplePairsData.append(euclidean_distance(Sample_List.iloc[Sample,1:-1],Sample_List.iloc[Alternate_Sample,1:-1]))

        temp_df = pd.DataFrame([SamplePairsData])
        euclidean_feature_dataframe = pd.concat([euclidean_feature_dataframe, temp_df], ignore_index=False)
        print(Sample)
    return(euclidean_feature_dataframe)
NewDataFrame=euclidean_feature_generator(disease_data)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [123]:
print(len((NewDataFrame).columns))
print(NewDataFrame) #Lets say we generate training data where we only want to know pairwise samples for a random assortment of known sample features
# We would have to keep the same shape for our tensor data, so we would need to input the value of 0 for pairwise samples we would not use
#lets say we select ten samples for our training data, lets say 5 is the beginning one, we would only input the other pairs avaliable
#So 5-10, 5-14, 5-16, 5-40, 5-90 we would use iloc[column,row] and create a function which goes if row in trainingdata: newdf iloc[column,row]
#a loop within a loop for row in number of rows; for column in number of columns if column in columnlist: newdf.iloc[column,row]=df.iloc[column,row]
#else newdf.iloc[column,row]=0
#else newdf iloc[column,row]=0
#This would create a new dataframe for all our samples of interest, get the value from our old dataframe then 

548
             0             1             2             3             4     
0       0.000000  26211.423872  11754.489940  14410.024696  11158.738892  \
0   26211.423872      0.000000  14503.615991  30173.621708  22611.253240   
0   11754.489940  14503.615991      0.000000  18577.114360  11012.677328   
0   14410.024696  30173.621708  18577.114360      0.000000  17455.058164   
0   11158.738892  22611.253240  11012.677328  17455.058164      0.000000   
..           ...           ...           ...           ...           ...   
0   11007.241378  17516.217235   6037.039245  19445.085441  14355.046650   
0   10222.886864  22587.551556  10555.243220  17493.876188   6122.052434   
0    9933.081490  27880.617640  14947.009001  16052.607529   8317.623878   
0    6482.703518  21774.599105   8178.868512  15262.969828  12111.568337   
0    6015.708451  21417.536566   7685.654620  16450.753924  12164.451843   

             5             6             7             8             9     
0   222

# Part 5 : Creation of a machine learning model

# Part 6 : Analysis of models efficiency and loss values

# Part 7 : Log of models